In [3]:
from os import walk
import pickle
import pandas as pd
import numpy as np

In [ ]:
class EnsembleModel:
    def __init__(self, anomaly_frac = 0.1, vote_threshold = None):
        self.anomaly_frac = anomaly_frac
        self.vote_threshold = vote_threshold
        self.models = []
        self.model_weigths = []

    def load_models(self, model_filepath):
        for (root, dirs, file) in walk(model_filepath):
            for f in file:
                if '.pkl' in f:
                    try:
                        self.models.append(pickle.load(open(f'{root}{f}', 'rb')))
                    except:
                        continue
        if not self.model_weigths:
            self.model_weigths = np.ones(len(self.models))

    def set_model_weights(self, weights):
        if len(weights) == len(self.models):
            self.model_weigths = weights
        else:
            raise Exception('Number of model weights must be equal to the number of models.') 
    
    def predict(self, X):
        predictions = []
        for m in self.models:
            predictions.append(m.predict(X))
        
        votes = np.zeros(X.shape[0])
        for i, p in enumerate(predictions):
            votes = votes + (p * self.model_weigths[i])
        
        if self.vote_threshold:
            return (votes >= self.vote_threshold)
        else:
            n_anomalies = np.ceil(self.anomaly_frac * X.shape[0])
            most_votes_ind = np.sort(np.argpartition(votes, -n_anomalies)[-n_anomalies:])
            return X[most_votes_ind]

    